In [1]:
!git clone https://github.com/rafikg/CEAL.git

Cloning into 'CEAL'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 395 (delta 11), reused 46 (delta 8), pack-reused 342
Receiving objects: 100% (395/395), 241.62 KiB | 2.12 MiB/s, done.
Resolving deltas: 100% (181/181), done.


In [2]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from typing import Optional, Callable, Union
from torch.utils.data import Dataset
import os
import glob
import cv2
import numpy as np
import torch
import logging


In [3]:
from typing import Tuple

def get_high_confidence_samples(pred_prob: np.ndarray,
                                delta: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Select high confidence samples from `D^U` whose entropy is smaller than
     the threshold
    `delta`.

    Parameters
    ----------
    pred_prob : np.ndarray
        prediction probability of x_i with dimension (batch x n_class)
    delta : float
        threshold

    Returns
    -------
    np.array with dimension (K x 1)  containing the indices of the K
        most informative samples.
    np.array with dimension (K x 1) containing the predicted classes of the
        k most informative samples
    """
    _, eni = entropy(pred_prob=pred_prob, k=len(pred_prob))
    hcs = eni[eni[:, 2] < delta]
    return hcs[:, 0].astype(np.int32), hcs[:, 1].astype(np.int32)


def get_uncertain_samples(pred_prob: np.ndarray, k: int,
                          criteria: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Get the K most informative samples based on the criteria
    Parameters
    ----------
    pred_prob : np.ndarray
        prediction probability of x_i with dimension (batch x n_class)
    k: int
    criteria: str
        `cl` : least_confidence()
        `ms` : margin_sampling()
        `en` : entropy

    Returns
    -------
    tuple(np.ndarray, np.ndarray)
    """
    if criteria == 'cl':
        uncertain_samples = least_confidence(pred_prob=pred_prob, k=k)
    elif criteria == 'ms':
        uncertain_samples = margin_sampling(pred_prob=pred_prob, k=k)
    elif criteria == 'en':
        uncertain_samples = entropy(pred_prob=pred_prob, k=k)
    else:
        raise ValueError('criteria {} not found !'.format(criteria))
    return uncertain_samples


In [4]:
# Authors rafik gouiaa <rafikgouiaaphd@gmail.com>, ...
from typing import Tuple
import numpy as np


def least_confidence(pred_prob: np.ndarray, k: int) -> Tuple[np.ndarray,
                                                             np.ndarray]:
    f"""
    Rank all the unlabeled samples in an ascending order according to
    equation 2

    Parameters
    ----------
    pred_prob : prediction probability of x_i with dimension (batch x n_class)
    k : int
        most informative samples
    Returns
    -------
    np.array with dimension (K x 1) containing the indices of the K
        most informative samples.
    np.array with dimension (K x 3) containing the indices, the predicted class
        and the `lc` of the k most informative samples
        column 1: indices
        column 2: predicted class.
        column 3: lc
    """
    assert np.round(pred_prob.sum(1).sum()) == pred_prob.shape[
        0], "pred_prob is not " \
            "a probability" \
            " distribution"
    assert 0 < k <= pred_prob.shape[0], "invalid k value k should be >0 &" \
                                        "k <=  pred_prob.shape[0"
    # Get max probabilities prediction and its corresponding classes
    most_pred_prob, most_pred_class = np.max(pred_prob, axis=1), np.argmax(
        pred_prob, axis=1)
    size = len(pred_prob)
    lc_i = np.column_stack(
        (list(range(size)), most_pred_class, most_pred_prob))
    # sort lc_i in ascending order
    lc_i = lc_i[lc_i[:, -1].argsort()]

    return lc_i[:k, 0].astype(np.int32), lc_i[:k]


def margin_sampling(pred_prob: np.ndarray, k: int) -> Tuple[np.ndarray,
                                                            np.ndarray]:
    f"""
    Rank all the unlabeled samples in an ascending order according to the
    equation 3
    ----------
    pred_prob : np.ndarray
        prediction probability of x_i with dimension (batch x n_class)
    k : int
        most informative samples

    Returns
    -------
    np.array with dimension (K x 1)  containing the indices of the K
        most informative samples.
    np.array with dimension (K x 3) containing the indices, the predicted class
        and the `ms_i` of the k most informative samples
        column 1: indices
        column 2: predicted class.
        column 3: margin sampling
    """
    assert np.round(pred_prob.sum(1).sum()) == pred_prob.shape[
        0], "pred_prob is not " \
            "a probability" \
            " distribution"
    assert 0 < k <= pred_prob.shape[0], "invalid k value k should be >0 &" \
                                        "k <=  pred_prob.shape[0"
    # Sort pred_prob to get j1 and j2
    size = len(pred_prob)
    margin = np.diff(np.abs(np.sort(pred_prob, axis=1)[:, ::-1][:, :2]))
    pred_class = np.argmax(pred_prob, axis=1)
    ms_i = np.column_stack((list(range(size)), pred_class, margin))

    # sort ms_i in ascending order according to margin
    ms_i = ms_i[ms_i[:, 2].argsort()]

    # the smaller the margin  means the classifier is more
    # uncertain about the sample
    return ms_i[:k, 0].astype(np.int32), ms_i[:k]


def entropy(pred_prob: np.ndarray, k: int) -> Tuple[np.ndarray, np.ndarray]:
    f"""
    Rank all the unlabeled samples in an descending order according to
    the equation 4

    Parameters
    ----------
    pred_prob : np.ndarray
        prediction probability of x_i with dimension (batch x n_class)
    k : int

    Returns
    -------
    np.array with dimension (K x 1)  containing the indices of the K
        most informative samples.
    np.array with dimension (K x 3) containing the indices, the predicted class
        and the `en_i` of the k most informative samples
        column 1: indices
        column 2: predicted class.
        column 3: entropy

    """
    # calculate the entropy for the pred_prob
    assert np.round(pred_prob.sum(1).sum()) == pred_prob.shape[
        0], "pred_prob is not " \
            "a probability" \
            " distribution"
    assert 0 < k <= pred_prob.shape[0], "invalid k value k should be >0 &" \
                                        "k <=  pred_prob.shape[0"
    size = len(pred_prob)
    entropy_ = - np.nansum(pred_prob * np.log(pred_prob), axis=1)
    pred_class = np.argmax(pred_prob, axis=1)
    en_i = np.column_stack((list(range(size)), pred_class, entropy_))

    # Sort en_i in descending order
    en_i = en_i[(-1 * en_i[:, 2]).argsort()]
    return en_i[:k, 0].astype(np.int32), en_i[:k]


In [5]:
# Authors: rafik gouiaa <rafikgouiaaphd@gmail.com>, ...

from typing import Optional, Callable

from torchvision.models import alexnet
from torch.utils.data import DataLoader
from torch.nn.functional import softmax
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch
import torch.optim as Optimizer
from tqdm import tqdm

class AlexNet(object):
    """
    Encapsulate the pretrained alexnet model
    Parameters
    ----------
    n_classes : int, default(10)
        the new number of classes
    device: Optional[str] 'cuda' or 'cpu', default(None)
            if None: cuda will be used if it is available
    """

    def __init__(self, n_classes: int = 10, device: Optional[str] = None):

        self.n_classes = n_classes
        self.model = alexnet(pretrained=True, progress=True)

        ## freeze_all_layers
        for param in self.model.parameters():
            param.requires_grad = False

        ## change last layer to accept n_classes instead of 1000 classes
        self.model.classifier[6] = nn.Linear(4096, self.n_classes)

        ## Add softmax layer to alexnet model
        self.model = nn.Sequential(self.model, nn.LogSoftmax(dim=1))

        if device is None:
            self.device = torch.device(
                "cuda:0" if torch.cuda.is_available() else "cpu")
        print('The code is running on {} '.format(self.device))

        # print(self.model)



    def __train_one_epoch(self, train_loader: DataLoader,
                          optimizer: Optimizer,
                          criterion: Callable,
                          valid_loader: DataLoader = None,
                          epoch: int = 0,
                          each_batch_idx: int = 300) -> None:
        """
        Train alexnet for one epoch
        Parameters
        ----------
        train_loader : DataLoader
        criterion :  Callable
        optimizer : Optimizer (torch.optim)
        epoch : int
        each_batch_idx : int
            print training stats after each_batch_idx

        Returns
        -------
        None
        """
        train_loss = 0
        data_size = 0
        print("Training ...")
        for batch_idx, sample_batched in tqdm(enumerate(train_loader)):
            # load data and label
            data, label = sample_batched['image'], sample_batched['label']

            # convert data and label to be compatible with the device
            data = data.to(self.device)
            data = data.float()
            label = label.to(self.device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # run forward
            pred_prob = self.model(data)

            # calculate loss
            loss = criterion(pred_prob, label)

            # calculate gradient (backprop)
            loss.backward()

            # total train loss
            train_loss += loss.item()
            data_size += label.size(0)

            # update weights
            optimizer.step()

            if batch_idx % each_batch_idx == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data),
                    len(train_loader.sampler.indices),
                    100. * batch_idx / len(train_loader.sampler.indices),
                    loss.item()))
        if valid_loader:
            acc = self.evaluate(test_loader=valid_loader)
            print('Accuracy on the valid dataset {}'.format(acc))

        print('====> Epoch: {} Average loss: {:.4f}'.
              format(epoch,
                     train_loss / data_size))

    def train(self, epochs: int, train_loader: DataLoader,
              valid_loader: DataLoader = None) -> None:
        """
        Train alexnet for several epochs
        Parameters
        ----------
        epochs : int
            number of epochs
        train_loader:  DataLoader
            training set
        valid_loader : DataLoader, Optional

        Returns
        -------
        None
        """
        self.model.to(self.device)
        self.model.train()
        optimizer = optim.SGD(
            filter(lambda p: p.requires_grad, self.model.parameters()),
            lr=0.001, momentum=0.9)

        criterion = nn.CrossEntropyLoss()
        for epoch in range(epochs):
            self.__train_one_epoch(train_loader=train_loader,
                                   optimizer=optimizer,
                                   criterion=criterion,
                                   valid_loader=valid_loader,
                                   epoch=epoch
                                   )

    def evaluate(self, test_loader: DataLoader) -> float:
        """
        Calaculate alexnet accuracy on test data
        Parameters
        ----------
        test_loader: DataLoader

        Returns
        -------
        accuracy: float
        """
        correct = 0
        total = 0
        print("Evaluation ...")
        with torch.no_grad():
            for batch_idx, sample_batched in tqdm(enumerate(test_loader)):
                data, labels = sample_batched['image'], \
                               sample_batched['label']
                data = data.to(self.device)
                data = data.float()
                labels = labels.to(self.device)
                outputs = self.model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        return 100 * correct / total

    def predict(self, test_loader):
        """
        Run the inference pipeline on the test_loader data
        Parameters
        ----------
        test_loader: DataLoader
            test data

        Returns
        -------

        """
        self.model.eval()
        self.model.to(self.device)
        predict_results = np.empty(shape=(0, 10))
        print("Prediction on Unlabelled Data ...")
        with torch.no_grad():
            for batch_idx, sample_batched in tqdm(enumerate(test_loader)):
                data, _ = sample_batched['image'], \
                          sample_batched['label']
                data = data.to(self.device)
                data = data.float()
                outputs = self.model(data)
                outputs = softmax(outputs)
                predict_results = np.concatenate(
                    (predict_results, outputs.cpu().numpy()))
        return predict_results


## Dataset

In [6]:
pwd

'/content'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
mkdir data

In [9]:
!git clone https://github.com/teavanist/MNIST-JPG.git

Cloning into 'MNIST-JPG'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 29 (delta 7), reused 27 (delta 7), pack-reused 1
Receiving objects: 100% (29/29), 17.53 MiB | 13.63 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [10]:
!unzip "/content/MNIST-JPG/MNIST Dataset JPG format.zip" -d /content/dataMnist

Streaming output truncated to the last 5000 lines.
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/18241.jpg  
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/18251.jpg  
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/1826.jpg  
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/18277.jpg  
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/18296.jpg  
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/183.jpg  
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/18350.jpg  
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/18355.jpg  
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/18374.jpg  
 extracting: /content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training/9/18377.jpg  
 extracting:

In [11]:
!unzip "/content/data/train-images.idx3-ubyte" -d /content/data/train-images

unzip:  cannot find or open /content/data/train-images.idx3-ubyte, /content/data/train-images.idx3-ubyte.zip or /content/data/train-images.idx3-ubyte.ZIP.


In [ ]:
# os.system("rm -rf /content/data/256_ObjectCategories/257.clutter")

0

In [ ]:
# !pip install patool
# import patoolib


In [ ]:
# patoolib.extract_archive("/content/drive/MyDrive/Data/project/mnist/MNIST_ORG.zip",outdir="/content/data")

In [12]:
%%writefile divide_data.sh

# This script is to divide the dataset into 80% 20%
echo "If you run this script on linux, change gshuf with shuf "
base_folder="/content/data/mnist_test_custom/"

mkdir -p "$base_folder"

for dir in "/content/data/trainingSet"*
do

  subdir=$(basename "$dir")

  mkdir -p "$base_folder$subdir"

  count=$(ls "$dir" | wc -l)

  tenpercent=$(expr $count '*' 20 '/' 100)

  ls "$dir" | shuf -n "$tenpercent" | xargs -I {} mv "$dir"/{} "$base_folder$subdir"

done


Writing divide_data.sh


## to split the data

In [13]:
!bash divide_data.sh

If you run this script on linux, change gshuf with shuf 
ls: cannot access '/content/data/trainingSet*': No such file or directory
ls: cannot access '/content/data/trainingSet*': No such file or directory


In [14]:

class mnistDataset(Dataset):
    """
    Encapsulate Caltech256 torch.utils.data.Dataset

    Parameters
    ----------
    root_dir : str
        Path to the dataset directory.

    transform : Callable,
        A transform function that takes the original image and
        return a transformed version.

    Attributes
    ----------
    data : list
        list of images files names
    labels : list
        list of integers (labels)
    """

    def __init__(self, root_dir,
                 transform: Optional[Callable] = None):

        self.root_dir = os.path.expanduser(root_dir)
        self.transform = transform
        self.data = []
        self.labels = []
        self._classes = 10

        # load data and labels
        for cat in range(0, self._classes):
            cat_dir = glob.glob(
                os.path.join(self.root_dir, str(cat)))[0]

            for img_file in glob.glob(os.path.join(cat_dir, '*.jpg')):
                self.data.append(img_file)
                self.labels.append(cat)

    def __getitem__(self, idx: int) -> dict:
        """
        Get the idx element

        Parameters
        ----------
        idx : int
           the index of the element


        Returns
        -------
        sample: dict[str, Any]
        """
        img, label = self.data[idx], self.labels[idx]
        label = torch.tensor(label,dtype = torch.long)
        img = cv2.imread(img)
        img = img[:, :, ::-1]
        img = self.img_normalize(img).transpose(2, 0, 1)
        sample = {'image': img, 'label': label}

        if self.transform:
            sample['image'] = self.transform(torch.tensor(sample['image']))

        return sample

    def __len__(self):

        return len(self.data)

    @staticmethod
    def img_normalize(img):
        img = (img / 255.0)

        return img

In [15]:
def update_threshold(delta: float, dr: float, t: int) -> float:
    """
    Update the selection threshold of high confidence samples
    Parameters
    ----------
    delta
    dr
    t

    Returns
    -------

    """
    if t > 0:
        delta = delta - dr * t
    return delta


In [16]:
import os
# len(os.listdir("/content/data/256_ObjectCategories"))

In [17]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import torch

def ceal_learning_algorithm(du: DataLoader,
                            dl: DataLoader,
                            dtest: DataLoader,
                            k: int = 1000,
                            delta_0: float = 0.00005,
                            dr: float = 0.000033,
                            t: int = 1,
                            epochs: int = 15,
                            criteria: str = 'cl',
                            max_iter: int = 25):
    """
    Algorithm1 : Learning algorithm of CEAL.
    For simplicity, I used the same notation in the paper.
    Parameters
    ----------
    du: DataLoader
        Unlabeled samples
    dl : DataLoader
        labeled samples
    dtest : DataLoader
        test data
    k: int, (default = 1000)
        uncertain samples selection
    delta_0: float
        hight confidence samples selection threshold
    dr: float
        threshold decay
    t: int
        fine-tuning interval
    epochs: int
    criteria: str
    max_iter: int
        maximum iteration number.

    Returns
    -------

    """
    print('Initial configuration: len(du): {}, len(dl): {} '.format(
        len(du.sampler.indices),
        len(dl.sampler.indices)))

    # Create the model
    model = AlexNet(n_classes=10, device=None)

    # Initialize the model
    print('Intialize training the model on `dl` and test on `dtest`')

    model.train(epochs=epochs, train_loader=dl, valid_loader=None)

    # Evaluate model on dtest
    acc = model.evaluate(test_loader=dtest)

    print('====> Initial accuracy: {} '.format(acc))

    for iteration in range(max_iter):

        print('Iteration: {}: run prediction on unlabeled data '
                    '`du` '.format(iteration))

        pred_prob = model.predict(test_loader=du)

        # get k uncertain samples
        uncert_samp_idx, _ = get_uncertain_samples(pred_prob=pred_prob, k=k,
                                                   criteria=criteria)

        # get original indices
        uncert_samp_idx = [du.sampler.indices[idx] for idx in uncert_samp_idx]

        # add the uncertain samples selected from `du` to the labeled samples
        #  set `dl`
        dl.sampler.indices.extend(uncert_samp_idx)

        print(
            'Update size of `dl`  and `du` by adding uncertain {} samples'
            ' in `dl`'
            ' len(dl): {}, len(du) {}'.
            format(len(uncert_samp_idx), len(dl.sampler.indices),
                   len(du.sampler.indices)))

        # get high confidence samples `dh`
        hcs_idx, hcs_labels = get_high_confidence_samples(pred_prob=pred_prob,
                                                          delta=delta_0)
        # get the original indices
        hcs_idx = [du.sampler.indices[idx] for idx in hcs_idx]

        # remove the samples that already selected as uncertain samples.
        hcs_idx = [x for x in hcs_idx if
                   x not in list(set(uncert_samp_idx) & set(hcs_idx))]

        # add high confidence samples to the labeled set 'dl'

        # (1) update the indices
        dl.sampler.indices.extend(hcs_idx)
        # (2) update the original labels with the pseudo labels.
        for idx in range(len(hcs_idx)):
            dl.dataset.labels[hcs_idx[idx]] = hcs_labels[idx]
        print(
            'Update size of `dl`  and `du` by adding {} hcs samples in `dl`'
            ' len(dl): {}, len(du) {}'.
            format(len(hcs_idx), len(dl.sampler.indices),
                   len(du.sampler.indices)))

        if iteration % t == 0:
            print('Iteration: {} fine-tune the model on dh U dl'.
                        format(iteration))
            model.train(epochs=epochs, train_loader=dl)

            # update delta_0
            delta_0 = update_threshold(delta=delta_0, dr=dr, t=iteration)

        # remove the uncertain samples from the original `du`
        print('remove {} uncertain samples from du'.
                    format(len(uncert_samp_idx)))
        for val in uncert_samp_idx:
            du.sampler.indices.remove(val)

        acc = model.evaluate(test_loader=dtest)
        print(
            "Iteration: {}, len(dl): {}, len(du): {},"
            " len(dh) {}, acc: {} ".format(
                iteration, len(dl.sampler.indices),
                len(du.sampler.indices), len(hcs_idx), acc))


if __name__ == "__main__":

    dataset_train = mnistDataset(
        root_dir="/content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - training",
        transform=transforms.Compose([transforms.ToPILImage(),
        # transforms.RandomResizedCrop((224,224)),
        # transforms.RandomResizedCrop((64,64)),
        transforms.Resize((150,150)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor()]))

    dataset_test = mnistDataset(
        root_dir="/content/dataMnist/MNIST Dataset JPG format/MNIST - JPG - testing",
        transform=transforms.Compose([transforms.ToPILImage(),
        # transforms.RandomResizedCrop((224,224)),
        # transforms.RandomResizedCrop((64,64)),
        transforms.Resize((150,150)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor()]))

    # Creating data indices for training and validation splits:
    random_seed = 123
    validation_split = 0.1  # 10%
    shuffling_dataset = True
    batch_size = 16
    dataset_size = len(dataset_train)

    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))

    if shuffling_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    du = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                                     sampler=train_sampler, num_workers=4)
    dl = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                                     sampler=valid_sampler, num_workers=4)
    dtest = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,
                                        num_workers=4)

    ceal_learning_algorithm(du=du, dl=dl, dtest=dtest)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also

Initial configuration: len(du): 54000, len(dl): 6000 


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 136MB/s]


The code is running on cuda:0 
Intialize training the model on `dl` and test on `dtest`
Training ...


9it [00:06,  1.88it/s]

...

Train Epoch: 14 [28800/32731 (5%)]	Loss: 0.173485


2046it [00:48, 41.94it/s]


====> Epoch: 14 Average loss: 0.0421
remove 1000 uncertain samples from du
Evaluation ...


625it [00:15, 40.12it/s]

Iteration: 24, len(dl): 32731, len(du): 29000, len(dh) 0, acc: 91.35 


In [ ]:
len(dataset_train)

42000